# Simple Variational AutoEncoder training and inference example

**Description**: in this notebook, we showcase the training process and inference capabilities of a simple variational auto-encoder model.

## Imports, definitions and setup

The first block is needed only when the current environment doesn't have the `dlproject` package installed.
Therefore, if you already cloned the whole repository and run the `pip install -e .` command, you can skip the first block.

If you're running this notebook only on a Jupyter server, run the first block as well in order to obtain the necessary dependencies.

In [ ]:
!git clone https://github.com/peiva-git/deep_learning_project.git
%cd deep_learning_project
!pip install -e .

In [2]:
import dlproject as dlp
import tensorflow as tf

import os.path

## Load the MNIST dataset

In [3]:
dataset_builder = dlp.data.MNISTDatasetBuilder()
dataset_builder.preprocess_dataset_simple_vae()
train_x, test_x = dataset_builder.train_x, dataset_builder.test_x
train_y, test_y = dataset_builder.train_y, dataset_builder.test_y

## Instantiate the model

In [4]:
vae_wrapper = dlp.models.SimpleVAE(input_dim=28 * 28, latent_dim=2)
vae_model = vae_wrapper.vae
vae_model.compile(optimizer=tf.keras.optimizers.Adam())

2023-11-06 18:18:37.063045: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-06 18:18:37.069650: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-06 18:18:37.069808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Train the model

Train the instantiated model on the MNIST dataset.

This block also saves a backup and a checkpoint every 20 epochs, so that you can automatically resume the training if it gets interrupted.

In [ ]:
if not os.path.exists(os.path.join(os.getcwd(), 'output', 'training-callback-results', f'{vae_model.name}_mnist')):
    os.makedirs(os.path.join(os.getcwd(), 'output', 'training-callback-results', f'{vae_model.name}_mnist', 'backup'))
    os.makedirs(os.path.join(os.getcwd(), 'output', 'training-callback-results', f'{vae_model.name}_mnist', 'model_checkpoints'))

model_dir_path = os.path.join(os.getcwd(), 'output', 'training-callback-results', f'{vae_model.name}_mnist')

vae_model.fit(
    train_x, train_x,
    epochs=100,
    batch_size=32,
    validation_data=(test_x, test_x),
    callbacks=[
        tf.keras.callbacks.BackupAndRestore(
            backup_dir=os.path.join(model_dir_path, 'backup'),
            save_freq=37500 # 20 * 1875, each 20 epochs
        ),
        tf.keras.callbacks.ModelCheckpoint(model_dir_path, 'model_checkpoints', save_freq=37500)
    ]
)

Epoch 1/100


2023-11-06 18:18:43.778940: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-06 18:18:44.098705: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f07508bf7f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-06 18:18:44.098729: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1050, Compute Capability 6.1
2023-11-06 18:18:44.102634: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-06 18:18:44.113523: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-06 18:18:44.183335: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 [==============================] - 8s 3ms/step - loss: 185.6512 - val_loss: 167.0640
Epoch 2/100
1875/1875 [==============================] - 5s 3ms/step - loss: 164.8788 - val_loss: 163.1181
Epoch 3/100
1875/1875 [==============================] - 5s 3ms/step - loss: 161.7603 - val_loss: 160.6381
Epoch 4/100
1875/1875 [==============================] - 5s 3ms/step - loss: 159.6979 - val_loss: 159.1065
Epoch 5/100
1875/1875 [==============================] - 6s 3ms/step - loss: 158.2439 - val_loss: 157.9134
Epoch 6/100
1875/1875 [==============================] - 5s 3ms/step - loss: 157.0735 - val_loss: 156.7323
Epoch 7/100
1875/1875 [==============================] - 5s 3ms/step - loss: 156.0399 - val_loss: 155.9010
Epoch 8/100
1875/1875 [==============================] - 5s 3ms/step - loss: 155.0881 - val_loss: 154.9742
Epoch 9/100
1875/1875 [==============================] - 5s 3ms/step - loss: 154.2814 - val_loss: 154.2753
Epoch 10/100
1875/1875 [=========================

INFO:tensorflow:Assets written to: /home/peiva/PycharmProjects/deep_learning_project/output/training-callback-results/vae_mlp_mnist/assets


1875/1875 [==============================] - 6s 3ms/step - loss: 149.3325 - val_loss: 149.8401
Epoch 21/100
1875/1875 [==============================] - 5s 3ms/step - loss: 149.0473 - val_loss: 149.4780
Epoch 22/100
1875/1875 [==============================] - 5s 3ms/step - loss: 148.7963 - val_loss: 149.3871
Epoch 23/100
1875/1875 [==============================] - 6s 3ms/step - loss: 148.5472 - val_loss: 149.1288
Epoch 24/100
1875/1875 [==============================] - 5s 3ms/step - loss: 148.3611 - val_loss: 148.9132
Epoch 25/100
1875/1875 [==============================] - 5s 3ms/step - loss: 148.1432 - val_loss: 148.6704
Epoch 26/100
1875/1875 [==============================] - 5s 3ms/step - loss: 147.9358 - val_loss: 148.6243
Epoch 27/100
1875/1875 [==============================] - 5s 3ms/step - loss: 147.7985 - val_loss: 148.8833
Epoch 28/100
1875/1875 [==============================] - 5s 3ms/step - loss: 147.5800 - val_loss: 148.5708
Epoch 29/100
1875/1875 [=================

...

1875/1875 [==============================] - 6s 3ms/step - loss: 146.0429 - val_loss: 147.0916
Epoch 41/100
1875/1875 [==============================] - 5s 3ms/step - loss: 146.0246 - val_loss: 147.3248
Epoch 42/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.9033 - val_loss: 146.8750
Epoch 43/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.8194 - val_loss: 147.2118
Epoch 44/100
1875/1875 [==============================] - 6s 3ms/step - loss: 145.6967 - val_loss: 147.1136
Epoch 45/100
1875/1875 [==============================] - 6s 3ms/step - loss: 145.6492 - val_loss: 146.9806
Epoch 46/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.5205 - val_loss: 146.7894
Epoch 47/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.4770 - val_loss: 146.7543
Epoch 48/100
1875/1875 [==============================] - 5s 3ms/step - loss: 145.4001 - val_loss: 146.8427
Epoch 49/100
1875/1875 [=================

INFO:tensorflow:Assets written to: /home/peiva/PycharmProjects/deep_learning_project/output/training-callback-results/vae_mlp_mnist/assets


1875/1875 [==============================] - 6s 3ms/step - loss: 144.5690 - val_loss: 146.1395
Epoch 61/100
1875/1875 [==============================] - 5s 3ms/step - loss: 144.4914 - val_loss: 145.9302
Epoch 62/100
1875/1875 [==============================] - 5s 3ms/step - loss: 144.4557 - val_loss: 146.2572
Epoch 63/100
1875/1875 [==============================] - 5s 3ms/step - loss: 144.3932 - val_loss: 146.0884
Epoch 64/100
1875/1875 [==============================] - 5s 3ms/step - loss: 144.3462 - val_loss: 146.5727
Epoch 65/100
1875/1875 [==============================] - 5s 3ms/step - loss: 144.2747 - val_loss: 146.2657
Epoch 66/100
1875/1875 [==============================] - 5s 3ms/step - loss: 144.2360 - val_loss: 146.0807
Epoch 67/100
1875/1875 [==============================] - 5s 3ms/step - loss: 144.1621 - val_loss: 145.9100
Epoch 68/100
1875/1875 [==============================] - 6s 3ms/step - loss: 144.1387 - val_loss: 145.8492
Epoch 69/100
1875/1875 [=================

INFO:tensorflow:Assets written to: /home/peiva/PycharmProjects/deep_learning_project/output/training-callback-results/vae_mlp_mnist/assets


1875/1875 [==============================] - 6s 3ms/step - loss: 143.5260 - val_loss: 145.7112
Epoch 81/100
1875/1875 [==============================] - 5s 3ms/step - loss: 143.5025 - val_loss: 145.6325
Epoch 82/100
1875/1875 [==============================] - 5s 3ms/step - loss: 143.4673 - val_loss: 145.8889
Epoch 83/100
 976/1875 [==============>...............] - ETA: 2s - loss: 143.5051

## Save the trained model

Save the just trained model for later use.

In [ ]:
if not os.path.exists(os.path.join(os.getcwd(), 'output', 'models')):
    os.mkdir(os.path.join(os.getcwd(), 'output', 'models'))

vae_model.save(os.path.join(os.getcwd(), 'output', 'models', f'{vae_model.name}_mnist.keras'))
vae_wrapper.encoder.save(os.path.join(os.getcwd(), 'output', 'models', f'{vae_wrapper.encoder.name}_mnist.keras'))
vae_wrapper.decoder.save(os.path.join(os.getcwd(), 'output', 'models', f'{vae_wrapper.decoder.name}_mnist.keras'))

## Load the model

Instead of training the model, you can load it from a previously saved `.keras` file.

In [8]:
vae_model = tf.keras.saving.load_model(os.path.join(os.getcwd(), 'output', 'models', 'vae_mlp_mnist.keras'), safe_mode=False)
vae_encoder = tf.keras.saving.load_model(os.path.join(os.getcwd(), 'output', 'models', 'vae_encoder_mnist.keras'), safe_mode=False)
vae_decoder = tf.keras.saving.load_model(os.path.join(os.getcwd(), 'output', 'models', 'vae_decoder_mnist.keras'), safe_mode=False)

ValueError: Layer 'dense' expected 2 variables, but received 0 variables during loading. Expected: ['dense/kernel:0', 'dense/bias:0']

## Visualization

Display a scatter plot of the encoded test data.

In [22]:
dlp.data.show_mnist_scatter_plot(test_x, test_y, vae_wrapper.encoder, batch_size=32)

313/313 [==============================] - 0s 1ms/step


TypeError: list indices must be integers or slices, not tuple

<Figure size 600x600 with 0 Axes>

Display artificially generated digits.

In [23]:
dlp.data.show_latent_plane_sampling_digits(vae_wrapper.decoder)

ValueError: Exception encountered when calling layer 'vae_decoder' (type Functional).

Input 0 of layer "dense_3" is incompatible with the layer: expected min_ndim=2, found ndim=1. Full shape received: (2,)

Call arguments received by layer 'vae_decoder' (type Functional):
  • inputs=tf.Tensor(shape=(2,), dtype=float64)
  • training=None
  • mask=None